# Utilities

In [83]:
import cv2
import numpy as np

def mask_for_polygons(size, polygons):
    img_mask = np.zeros(size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

# Testing

In [84]:
from shapely.geometry import Polygon, MultiPolygon
#def test_mask_for_polygons():
polygon = Polygon([(0, 0), (1, 1), (4, 0), (0,3)])
polygons = MultiPolygon([polygon])
masked = mask_for_polygons((5,5), polygons)

In [85]:
masked

array([[1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

# Explore

## GDB file

In [86]:
from osgeo import ogr

In [87]:
gdbfile='/home/sravya/data/satellite/haiti/gdb/PDNA_HTI_2010_Atlas_of_Building_Damage_Assessment_UNOSAT_JRC_WB_v2.gdb'

In [88]:
#Get available drivers
cnt = ogr.GetDriverCount()
formatsList = []  # Empty List

for i in range(cnt):
    driver = ogr.GetDriver(i)
    driverName = driver.GetName()
    if not driverName in formatsList:
        formatsList.append(driverName)

formatsList.sort() # Sorting the messy list of ogr drivers

formatsList

['ARCGEN',
 'AVCBin',
 'AVCE00',
 'AeronavFAA',
 'AmigoCloud',
 'BNA',
 'CSV',
 'CSW',
 'Carto',
 'Cloudant',
 'CouchDB',
 'DGN',
 'DXF',
 'EDIGEO',
 'ESRI Shapefile',
 'ElasticSearch',
 'GFT',
 'GML',
 'GPKG',
 'GPSBabel',
 'GPSTrackMaker',
 'GPX',
 'GeoJSON',
 'GeoRSS',
 'Geoconcept',
 'HTF',
 'HTTP',
 'Idrisi',
 'Interlis 1',
 'Interlis 2',
 'JML',
 'JP2OpenJPEG',
 'KML',
 'MapInfo File',
 'Memory',
 'NAS',
 'ODS',
 'OGR_DODS',
 'OGR_GMT',
 'OGR_PDS',
 'OGR_SDTS',
 'OGR_VRT',
 'OSM',
 'OpenAir',
 'OpenFileGDB',
 'PCIDSK',
 'PDF',
 'PGDUMP',
 'PLSCENES',
 'PostgreSQL',
 'REC',
 'S57',
 'SEGUKOOA',
 'SEGY',
 'SQLite',
 'SUA',
 'SVG',
 'SXF',
 'Selafin',
 'TIGER',
 'UK .NTF',
 'VDV',
 'VFK',
 'WAsP',
 'WFS',
 'XLS',
 'XLSX',
 'XPlane',
 'netCDF']

In [89]:
# Seems like OpenFileGDB supports ESRI gdb files created with >ARGIS 9. 
# Ref: http://www.gdal.org/drv_openfilegdb.html
# Unitar page says (v. 9.3.1), so I am assuming we are good
driver = ogr.GetDriverByName("OpenFileGDB")
gdb_ds = driver.Open(gdbfile, 0)

In [90]:
# Check to see if shapefile is found.
if gdb_ds is None:
    print("Could not open %s" % (gdb_ds))
else:
    print('Opened %s' % (gdb_ds))

Opened <osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f821e0d86f0> >
Number of features : 294170


In [163]:
featsClassList = []
# parsing layers by index
for featsClass_idx in range(gdb_ds.GetLayerCount()):
    featsClass = gdb_ds.GetLayerByIndex(featsClass_idx)
    featsClassList.append(featsClass.GetName())

# sorting
featsClassList.sort()

# printing
for featsClass in featsClassList:
    print(featsClass)

UNOSAT_JRC_WB_Damage_Blds_v2


In [223]:
layer = gdb_ds.GetLayerByIndex(0)
featureCount = layer.GetFeatureCount()
print("Number of features : %d" % (featureCount))

Number of features : 294170


In [234]:
geom_names=[]
not_found_feats=[]
for i in range(0,layer.GetFeatureCount()):
    feat = layer.GetFeature(i)
    if feat is None:
        not_found_feats.append(i)
    else:
        geom = feat.GetGeometryRef()
        geom_names.append(geom.GetGeometryName())

In [236]:
len(not_found_feats)

36285

In [237]:
len(geom_names)

257885

In [239]:
Counter(geom_names)

Counter({'POINT': 257885})

### Explore gdb ds

In [91]:

gdb_ds

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f821e0d86f0> >

In [92]:
gdb_ds.GetLayerCount()

1

In [93]:
gdb_ds.GetLayerByIndex(0)

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x7f821e0d8360> >

In [94]:
gdb_ds.GetMetadata_Dict()

{}

In [95]:
gdb_ds.name

'/home/sravya/data/satellite/haiti/gdb/PDNA_HTI_2010_Atlas_of_Building_Damage_Assessment_UNOSAT_JRC_WB_v2.gdb'

## Shapefile

In [256]:
import os
shapefile = '/home/sravya/data/satellite/haiti/shapefile/'
driver = ogr.GetDriverByName('ESRI Shapefile')
shp_ds = driver.Open(daShapefile, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if shp_ds is None:
    print("Could not open %s" % (shp_ds))
else:
    print('Opened %s' % (shp_ds))
    layer = shp_ds.GetLayer()
    featureCount = layer.GetFeatureCount()
    print("Number of features : %d" % (featureCount))

Opened <osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f7fe255ad20> >
Number of features : 294170


### Explore features

In [176]:
layer = shp_ds.GetLayer()

In [101]:
layer.GetDescription()

'PDNA_HTI_2010_Atlas_of_Building_Damage_Assessment_UNOSAT_JRC_WB_v2'

In [102]:
layer.GetExtent()

(724423.8898999998, 814769.2341999998, 2015975.4824, 2119997.0555000007)

In [112]:
feat0=layer.GetFeature(0)

In [114]:
feat0.GetFieldCount()

17

In [115]:
feat1=layer.GetFeature(1)

In [116]:
feat1.GetFieldCount()

17

In [121]:
feat1.items()

{'Analysis': 'UNITAR/UNOSAT',
 'Commune': 'PORT-AU-PRINCE',
 'Damage_ID': 'GRADE 1 No Visible Damage',
 'Departemen': 'OUEST',
 'ID_Com': 111,
 'Id_dep': 1,
 'Landcover': 'Industrial Zone',
 'NOM_SECTIO': '6ème Turgeau',
 'No_section': 6,
 'OBJECTID': 2,
 'Population': 14760,
 'Section_': 'TURGEAU',
 'SensorDate': '2010/01/21',
 'Sensor_ID': 'Aerial Photography (Google)',
 'Site_ID': 'Building (General)',
 'Validation': 'Not yet field validated',
 'id_Section': '111-01'}

In [122]:
feat0.items()

{'Analysis': 'UNITAR/UNOSAT',
 'Commune': 'PORT-AU-PRINCE',
 'Damage_ID': 'GRADE 4 Very Heavy Damage',
 'Departemen': 'OUEST',
 'ID_Com': 111,
 'Id_dep': 1,
 'Landcover': 'High Density Built-Up Zone',
 'NOM_SECTIO': '6ème Turgeau',
 'No_section': 6,
 'OBJECTID': 1,
 'Population': 14760,
 'Section_': 'TURGEAU',
 'SensorDate': '2010/01/21',
 'Sensor_ID': 'Aerial Photography (Google)',
 'Site_ID': 'Building (General)',
 'Validation': 'Not yet field validated',
 'id_Section': '111-01'}

In [124]:
feat294170 = layer.GetFeature(294169)

In [125]:
feat294170.items()

{'Analysis': 'EC JRC',
 'Commune': None,
 'Damage_ID': 'GRADE 1 No Visible Damage',
 'Departemen': None,
 'ID_Com': 0,
 'Id_dep': 0,
 'Landcover': None,
 'NOM_SECTIO': None,
 'No_section': 0,
 'OBJECTID': 330454,
 'Population': 0,
 'Section_': None,
 'SensorDate': '2010/01/25',
 'Sensor_ID': 'Aerial Photography (Google)',
 'Site_ID': 'Building (General)',
 'Validation': 'Not yet field validated',
 'id_Section': None}

In [133]:
feat0.GetGeometryRef().Centroid()

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f821e0d8b40> >

In [127]:
feat294170.GetFID()

294169

In [128]:
feat294170.ExportToJson()

'{"type": "Feature", "geometry": {"type": "Point", "coordinates": [750382.6661999999, 2052557.8408]}, "properties": {"OBJECTID": 330454, "SensorDate": "2010/01/25", "ID_Com": 0, "NOM_SECTIO": null, "Id_dep": 0, "id_Section": null, "Departemen": null, "Commune": null, "No_section": 0, "Section_": null, "Population": 0, "Site_ID": "Building (General)", "Damage_ID": "GRADE 1 No Visible Damage", "Sensor_ID": "Aerial Photography (Google)", "Analysis": "EC JRC", "Validation": "Not yet field validated", "Landcover": null}, "id": 294169}'

In [113]:
feat0.geometry()

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f821e0d8570> >

In [172]:
feat1.GetField("Damage_ID")

'GRADE 1 No Visible Damage'

In [192]:
damage_ids=[]
for i in range(0,layer.GetFeatureCount()):
    damage_ids.append(layer.GetFeature(i).GetField("Damage_ID"))

In [193]:
damage_ids

['GRADE 4 Very Heavy Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 3 Substanial to Heavy Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 3 Substanial to Heavy Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible D

In [194]:
damage_set=set(damage_ids)

In [195]:
damage_set

{'GRADE 1 No Visible Damage',
 'GRADE 3 Substanial to Heavy Damage',
 'GRADE 4 Very Heavy Damage',
 'GRADE 5 Destruction',
 'Possible Damage'}

In [197]:
from collections import Counter
Counter(damage_ids)

Counter({'GRADE 1 No Visible Damage': 222253,
         'GRADE 3 Substanial to Heavy Damage': 15737,
         'GRADE 4 Very Heavy Damage': 24677,
         'GRADE 5 Destruction': 31489,
         'Possible Damage': 14})

In [215]:
feat1.ExportToJson()

'{"type": "Feature", "geometry": {"type": "Point", "coordinates": [780293.1216000002, 2053933.0095]}, "properties": {"OBJECTID": 2, "SensorDate": "2010/01/21", "ID_Com": 111, "NOM_SECTIO": "6\\u00e8me Turgeau", "Id_dep": 1, "id_Section": "111-01", "Departemen": "OUEST", "Commune": "PORT-AU-PRINCE", "No_section": 6, "Section_": "TURGEAU", "Population": 14760, "Site_ID": "Building (General)", "Damage_ID": "GRADE 1 No Visible Damage", "Sensor_ID": "Aerial Photography (Google)", "Analysis": "UNITAR/UNOSAT", "Validation": "Not yet field validated", "Landcover": "Industrial Zone"}, "id": 1}'

In [219]:
layer.GetGeometryColumn()

''

In [198]:
geom=feat1.GetGeometryRef()

In [ ]:
feat1.GetG

In [201]:
geom.GetCoordinateDimension()

2

In [202]:
geom.GetGeometryCount()

0

In [203]:
geom.GetGeometryName()

'POINT'

In [204]:
geom.GetGeometryType()

1

In [207]:
linear=geom.GetLinearGeometry()

In [209]:
geom.Area()

0.0

In [210]:
geom.Boundary()

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f7fe4b9aab0> >

In [240]:
centroid = geom.Centroid()

In [241]:
centroid.GetX()

760322.2548000002

In [242]:
centroid.GetY()

2017940.6239999998

In [212]:
geom.GetGeometryCount()

0

In [213]:
geom.GetPointCount()

1

In [243]:
geom.GetPoint()

(760322.2548000002, 2017940.6239999998, 0.0)

In [216]:
geom.GetX()

780293.1216000002

In [217]:
geom.GetY()

2053933.0095000006

In [221]:
geom_names=[]
for i in range(0,layer.GetFeatureCount()):
    feat = layer.GetFeature(i)
    geom = feat.GetGeometryRef()
    geom_names.append(geom.GetGeometryName())

In [222]:
Counter(geom_names)

Counter({'POINT': 294170})

## Filter features

In [260]:

layer.SetAttributeFilter("Damage_ID = 'GRADE 1 No Visible Damage'")

0

In [261]:
layer.GetFeatureCount()

222253

In [262]:
damage1=[]
#layer=shp_ds.GetLayer()
i=0
for feature in layer:
    damage1.append(feature.GetField("Damage_ID"))
    i=i+1

In [263]:
damage1

['GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 No Visible Damage',
 'GRADE 1 

In [264]:
print(i)

222253


### Spatial filter

In [265]:
wkt = "POLYGON ((780293.1216000002 2053933.0095000006,780393.1216000002 2053933.0095000006,780593.1216000002 2054933.0095000006,780293.1216000002 2054933.0095000006))"
layer.SetSpatialFilter(ogr.CreateGeometryFromWkt(wkt))


In [276]:
def setSpatialFilter(layer, x1, x2, y1, y2):
    wkt = "POLYGON (({} {},{} {},{} {},{} {}))".format(x1,y1, x1,y2, x2,y2, x2, y1)
    print(wkt)
    layer.SetSpatialFilter(ogr.CreateGeometryFromWkt(wkt))

In [277]:
#Extent = (724423.8898999998, 814769.2341999998, 2015975.4824, 2119997.0555000007)

In [280]:
setSpatialFilter(layer, 780000, 780050, 2050000, 2050050)

POLYGON ((780000 2050000,780000 2050050,780050 2050050,780050 2050000))


In [281]:
layer.GetFeatureCount()

33

### Get layer capabilities

In [282]:
capabilities = [
    ogr.OLCRandomRead,
    ogr.OLCSequentialWrite,
    ogr.OLCRandomWrite,
    ogr.OLCFastSpatialFilter,
    ogr.OLCFastFeatureCount,
    ogr.OLCFastGetExtent,
    ogr.OLCCreateField,
    ogr.OLCDeleteField,
    ogr.OLCReorderFields,
    ogr.OLCAlterFieldDefn,
    ogr.OLCTransactions,
    ogr.OLCDeleteFeature,
    ogr.OLCFastSetNextByIndex,
    ogr.OLCStringsAsUTF8,
    ogr.OLCIgnoreFields
]

print("Layer Capabilities:")
for cap in capabilities:
    print("  %s = %s" % (cap, layer.TestCapability(cap)))


Layer Capabilities:
  RandomRead = True
  SequentialWrite = False
  RandomWrite = False
  FastSpatialFilter = True
  FastFeatureCount = False
  FastGetExtent = True
  CreateField = False
  DeleteField = False
  ReorderFields = False
  AlterFieldDefn = False
  Transactions = False
  DeleteFeature = False
  FastSetNextByIndex = False
  StringsAsUTF8 = True
  IgnoreFields = True


# Raster data

In [285]:
rasterfile='/home/sravya/data/satellite/haiti/raster/EO1/EO1A0090472010015110P0_B01_L1T.TIF'
from osgeo import gdal
gtif = gdal.Open(rasterfile)
gtif.GetMetadata()

{'AREA_OR_POINT': 'Point',
 'TIFFTAG_RESOLUTIONUNIT': '1 (unitless)',
 'TIFFTAG_XRESOLUTION': '0',
 'TIFFTAG_YRESOLUTION': '0'}

In [286]:
gtif.GetDescription()

'/home/sravya/data/satellite/haiti/raster/EO1/EO1A0090472010015110P0_B01_L1T.TIF'

In [287]:
gtif.GetGCPCount()

0

In [288]:
gtif.GetGCPProjection()

''

In [289]:
gtif.GetGCPs()

()

In [290]:
gtif.GetGeoTransform()

(751795.0, 10.0, 0.0, 2111405.0, 0.0, -10.0)

In [291]:
gtif.RasterCount

1

In [353]:
gtif.GetProjection()

'PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]'

In [352]:
gtif.RasterXSize, gtif.RasterYSize

(6091, 13861)

In [332]:
gtif.GetSubDatasets()

[]

In [329]:
gtif.GetProjection()

'PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]'

In [294]:
band = gtif.GetRasterBand(1)

In [298]:
band.GetMinimum()

0.0

In [334]:
band.GetBlockSize()

[6091, 1]

In [335]:
band.GetMaskBand()

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x7f7fd9b432a0> >

In [336]:
band.GetOffset()

0.0

In [338]:
band.GetOverviewCount()

0

In [339]:
band.ComputeBandStats()

(670.5199409016161, 844.0635901387769)

In [341]:
band.DataType

3

In [343]:
dt = band.GetDataset()

In [346]:
dt.RasterCount

1

In [348]:
rat = band.GetDefaultRAT()

In [350]:
block = band.ReadRaster(0,0,10,10)

In [351]:
block

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [297]:
band.GetStatistics(True, True)

[0.0, 15046.0, 665.7157771005471, 831.5757358988171]

In [299]:
band.GetUnitType()

''

In [300]:
band.GetScale()

1.0

In [324]:
layer = gtif.GetLayer()

In [328]:
band.XSize, band.YSize

(6091, 13861)

In [331]:
band2 = gtif.GetRasterBand(0)
band2.XSize, band2.YSize

RuntimeError: /home/sravya/data/satellite/haiti/raster/EO1/EO1A0090472010015110P0_B01_L1T.TIF: GDALDataset::GetRasterBand(0) - Illegal band #


## Shape to Tiff

In [136]:
import gdal
pixelWidth = pixelHeight = 1 # depending how fine you want your raster
x_min, x_max, y_min, y_max = layer.GetExtent()
cols = int((x_max - x_min) / pixelHeight)
rows = int((y_max - y_min) / pixelWidth)
target_ds = gdal.GetDriverByName('GTiff').Create('temp.tif', cols, rows, 1, gdal.GDT_Byte) 
target_ds.SetGeoTransform((x_min, pixelWidth, 0, y_min, 0, pixelHeight))
band = target_ds.GetRasterBand(1)
NoData_value = 255
band.SetNoDataValue(NoData_value)
band.FlushCache()

In [145]:
gdal.RasterizeLayer(target_ds, [1], layer, options = ["ATTRIBUTE=Damage_ID"])  

0

In [146]:
import osr
target_dsSRS = osr.SpatialReference()
target_dsSRS.ImportFromEPSG(4326)
target_ds.SetProjection(target_dsSRS.ExportToWkt())

0

In [147]:
tif = gdal.Open('temp.tif')

In [149]:
tif.ReadAsArray()

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ..., 
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [ ]:
for feature in layer:
     feature.GetField("STATE_NAME")

In [153]:
import matplotlib.pyplot as plt
plt.imshow(tif)

TypeError: Image data can not convert to float

In [155]:
import tifffile as tiff

In [158]:
tif_img = tiff.imread('temp.tif')#.transpose([1, 2, 0])


/home/sravya/anaconda3/lib/python3.6/site-packages/tifffile/tifffile.py:2725: UserWarning: empty byte count
  warnings.warn("empty byte count")
/home/sravya/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:2903: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  'left=%s, right=%s') % (left, right))


(<matplotlib.figure.Figure at 0x7f7fe4b96b70>,
 <matplotlib.image.AxesImage at 0x7f7fe4b81da0>)

In [159]:
tif_img.shape

(104021, 90345)

In [160]:
tiff.imshow(tif_img[100:200,100:200])

/home/sravya/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:2903: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  'left=%s, right=%s') % (left, right))


(<matplotlib.figure.Figure at 0x7f7fe4a6e550>,
 <matplotlib.image.AxesImage at 0x7f7fe4a1f390>)

In [322]:
from osgeo import gdal, gdalnumeric, ogr, osr
#from PIL import Image
#import ImageDraw
import os, sys
gdal.UseExceptions()

# This function will convert the rasterized clipper shapefile
# to a mask for use within GDAL.
def imageToArray(i):
    """
    Converts a Python Imaging Library array to a
    gdalnumeric image.
    """
    a=gdalnumeric.fromstring(i.tostring(),'b')
    a.shape=i.im.size[1], i.im.size[0]
    return a

def arrayToImage(a):
    """
    Converts a gdalnumeric array to a
    Python Imaging Library Image.
    """
    i=Image.fromstring('L',(a.shape[1],a.shape[0]),
            (a.astype('b')).tostring())
    return i

def world2Pixel(geoMatrix, x, y):
  """
  Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
  the pixel location of a geospatial coordinate
  """
  ulX = geoMatrix[0]
  ulY = geoMatrix[3]
  xDist = geoMatrix[1]
  yDist = geoMatrix[5]
  rtnX = geoMatrix[2]
  rtnY = geoMatrix[4]
  pixel = int((x - ulX) / xDist)
  line = int((ulY - y) / xDist)
  return (pixel, line)

#
#  EDIT: this is basically an overloaded
#  version of the gdal_array.OpenArray passing in xoff, yoff explicitly
#  so we can pass these params off to CopyDatasetInfo
#
def OpenArray( array, prototype_ds = None, xoff=0, yoff=0 ):
    ds = gdal.Open( gdalnumeric.GetArrayFilename(array) )

    if ds is not None and prototype_ds is not None:
        if type(prototype_ds).__name__ == 'str':
            prototype_ds = gdal.Open( prototype_ds )
        if prototype_ds is not None:
            gdalnumeric.CopyDatasetInfo( prototype_ds, ds, xoff=xoff, yoff=yoff )
    return ds

def histogram(a, bins=range(0,256)):
  """
  Histogram function for multi-dimensional array.
  a = array
  bins = range of numbers to match
  """
  fa = a.flat
  n = gdalnumeric.searchsorted(gdalnumeric.sort(fa), bins)
  n = gdalnumeric.concatenate([n, [len(fa)]])
  hist = n[1:]-n[:-1]
  return hist

def stretch(a):
  """
  Performs a histogram stretch on a gdalnumeric array image.
  """
  hist = histogram(a)
  im = arrayToImage(a)
  lut = []
  for b in range(0, len(hist), 256):
    # step size
    step = reduce(operator.add, hist[b:b+256]) / 255
    # create equalization lookup table
    n = 0
    for i in range(256):
      lut.append(n / step)
      n = n + hist[i+b]
  im = im.point(lut)
  return imageToArray(im)

def main( shapefile_path, raster_path ):
    # Load the source data as a gdalnumeric array
    srcArray = gdalnumeric.LoadFile(raster_path)

    # Also load as a gdal image to get geotransform
    # (world file) info
    srcImage = gdal.Open(raster_path)
    geoTrans = srcImage.GetGeoTransform()

    # Create an OGR layer from a boundary shapefile
    shapef = ogr.Open(shapefile_path)
    lyr = shapef.GetLayer()#os.path.split( os.path.splitext( shapefile_path )[0] )[1] )
    #lyr = shapef.GetLayer( os.path.split( os.path.splitext( shapefile_path )[0] )[1] )
    poly = lyr.GetNextFeature()

    # Convert the layer extent to image pixel coordinates
    minX, maxX, minY, maxY = lyr.GetExtent()
    ulX, ulY = world2Pixel(geoTrans, minX, maxY)
    lrX, lrY = world2Pixel(geoTrans, maxX, minY)

    # Calculate the pixel size of the new image
    pxWidth = int(lrX - ulX)
    pxHeight = int(lrY - ulY)

    clip = srcArray[ulY:lrY, ulX:lrX]

    #
    # EDIT: create pixel offset to pass to new image Projection info
    #
    xoffset =  ulX
    yoffset =  ulY
    print("Xoffset, Yoffset = ( %f, %f )" % ( xoffset, yoffset ))

    # Create a new geomatrix for the image
    geoTrans = list(geoTrans)
    geoTrans[0] = minX
    geoTrans[3] = maxY

    # Map points to pixels for drawing the
    # boundary on a blank 8-bit,
    # black and white, mask image.
    points = []
    pixels = []
    geom = poly.GetGeometryRef()
    pts = geom.GetGeometryRef(0)
    for p in range(pts.GetPointCount()):
      points.append((pts.GetX(p), pts.GetY(p)))
    for p in points:
      pixels.append(world2Pixel(geoTrans, p[0], p[1]))
    rasterPoly = Image.new("L", (pxWidth, pxHeight), 1)
    rasterize = ImageDraw.Draw(rasterPoly)
    rasterize.polygon(pixels, 0)
    mask = imageToArray(rasterPoly)

    # Clip the image using the mask
    clip = gdalnumeric.choose(mask, \
        (clip, 0)).astype(gdalnumeric.uint8)

    # This image has 3 bands so we stretch each one to make them
    # visually brighter
    for i in range(3):
      clip[i,:,:] = stretch(clip[i,:,:])

    # Save new tiff
    #
    #  EDIT: instead of SaveArray, let's break all the
    #  SaveArray steps out more explicity so
    #  we can overwrite the offset of the destination
    #  raster
    #
    ### the old way using SaveArray
    #
    # gdalnumeric.SaveArray(clip, "OUTPUT.tif", format="GTiff", prototype=raster_path)
    #
    ###
    #
    gtiffDriver = gdal.GetDriverByName( 'GTiff' )
    if gtiffDriver is None:
        raise ValueError("Can't find GeoTiff Driver")
    gtiffDriver.CreateCopy( "OUTPUT.tif",
        OpenArray( clip, prototype_ds=raster_path, xoff=xoffset, yoff=yoffset )
    )

    # Save as an 8-bit jpeg for an easy, quick preview
    clip = clip.astype(gdalnumeric.uint8)
    gdalnumeric.SaveArray(clip, "OUTPUT.jpg", format="JPEG")

    gdal.ErrorReset()



In [323]:
main(shapefile, rasterfile)

Xoffset, Yoffset = ( -2737.000000, -859.000000 )


AttributeError: 'NoneType' object has no attribute 'GetPointCount'

In [97]:
>>> ds
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x02BB7038> >
>>> ds.GetLayer("buildings")
<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x02BB7050> >
>>> b = ds.GetLayer("buildings")
>>> sr = b.GetSpatialRef()
>>> sr
<osgeo.osr.SpatialReference; proxy of <Swig Object of type 'OSRSpatialReferenceShadow *' at 0x02BB7080> >
>>> sr.ExportToProj4()a

SyntaxError: invalid syntax (<ipython-input-97-b846d3e87e3c>, line 2)